# Hypothesis Testing

Importing libraries and the dataset

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
df = pd.read_feather('../datasets/stack_overflow.feather')
df.head()

# Hypothesis with one sample

A hypothesis is a statement about an unknown population parameter.

A hypothesis test is a test of two competing hypotheses:

* The null hypothesis (H0) is the existing idea.
* The alternative hypothesis (HA) is the new "challenger" idea of the researcher.

### Example 1

**Step 1**: Defitinion of the null and alternative hypotheses based on a parameter (in this case the mean).
* $H_0$: The proportion of data scientists starting programming as children is 35%
* $H_A$: The proportion of data scientists starting programming as children is greater than 35%

**Step 2**: Calculation of the z-score

$$z = \frac{\text{value}-\text{mean}}{\text{standard deviation}} = \frac{\text{sample stat}-\text{hypoth. param. value}}{\text{standard error}}$$

Where:

$$\text{standard error} = s_{\text{bootstrap distribution}}$$

In [10]:
# Generating a bootstrap distribution
comp_bootstrap_distribution = []
for i in range(5000):
    sample = df.sample(frac=1, replace=True)
    sample_bool = sample["age_first_code_cut"] == "child"
    comp_bootstrap_distribution.append(np.mean(sample_bool))

# Standard error - Standard deviation of the bootstrap distribution
std_error = np.std(comp_bootstrap_distribution, ddof=1)

# Mean of the sample
proportion_child_sample = (df["age_first_code_cut"] == "child").mean()

# Hypothesized value from the null hypothesis
proportion_child_hypothesis = 0.35

# Z-score
z_score = (proportion_child_sample - proportion_child_hypothesis) / std_error
print(z_score)

4.040952601350419


**Step 3:** Calculation of the p-value

In [6]:
from scipy.stats import norm

p_value = 1 - norm.cdf(z_score, loc=0, scale=1) # This is valid only when the alternative is greater than the null hypothesis.

# p_value = norm.cdf(z_score, loc=0, scale=1) # For left-tailed tests

print(p_value)

2.8421892364294266e-05


**Step 4:** Comparing the p-value against the significance level

The significance level of a hypothesis test ($\alpha$) is the threshold point for "beyond a reasonable doubt".

* If $p\le \alpha$, then we reject $H_0$
* But, if $p> \alpha$, we fail to reject $H_0$ (although it doesn't mean that $H_0$ is true)
  
$\alpha$ should be set **prior** to conducting the hypothesis test

In [7]:
alpha = 0.05
if p_value <= alpha:
    print("Null hypothesis rejected")
else:
    print("Failed to reject the null hypothesis")

Null hypothesis rejected


**Step 5:** Calculate the confidence intervals of the mean.

For a significance level of $\alpha$, it's common to choose a confidence interval level of $1-\alpha$.

* $\alpha = 0.05$ -> 95% confidence interval

In [9]:
lower = np.quantile(comp_bootstrap_distribution, 0.025)
upper = np.quantile(comp_bootstrap_distribution, 0.975)
print((lower, upper))

(0.3715170278637771, 0.41220698805838124)


### Possible errors

* Type I - False positive: we rejected the null hypothesis, when it was true.
* Type II - False negative: we failed to reject the null hypothesis, when it was false.

# Hypothesis with two samples

We are going to use the t-distribution or t-statistic, which is useful when we want to use a sample standard deviation in estimating a standard error (instead of using a bootstrapping distribution).

### Example 2

**Step 1**: Defitinion of the null and alternative hypotheses.

* $H_0$: The mean compensation (in USD) is the same for those that coded first as a child and those that coded first as an adult.

  $$H_0: \mu_{child}=\mu_{adult}$$
  $$H_0: \mu_{child}-\mu_{adult}=0$$
      
* $H_A$: The mean compensation (in USD) is greater for those that coded first as a child compared to those that coded first as an adult.
  
  $$H_A: \mu_{child}>\mu_{adult}$$
  $$H_A: \mu_{child}-\mu_{adult}>0$$

**Step 2**: Calculating the t-test

* Sample mean estimates the population mean
* $\bar x$: a sample mean
* $\bar x_{child}$: sample mean compensation for coding first as a child
* $\bar x_{adult}$: sample mean compensation for coding first as an adult
* $\bar x_{child}-\bar x_{adult}$: a test statistic

$$z=\frac{\text{sample stat}-\text{population parameter}}{\text{standard error}}$$

$$t=\frac{\text{difference in sample stats}-\text{difference in population parameters}}{\text{standard error}}$$

$$t=\frac{(\bar x_{child}-\bar x_{adult})-(\mu _{child}-\mu_{adult})}{SE(\bar x_{child}-\bar x_{adult})}$$

$$t=\frac{(\bar x_{child}-\bar x_{adult})-(0)}{SE(\bar x_{child}-\bar x_{adult})}$$

$$SE(\bar x_{child}-\bar x_{adult})\approx \sqrt{\frac{s^2_{child}}{n_{child}}+\frac{s^2_{adult}}{n_{adult}}}$$

$$t=\frac{(\bar x_{child}-\bar x_{adult})}{\sqrt{\frac{s^2_{child}}{n_{child}}+\frac{s^2_{adult}}{n_{adult}}}}$$

In [16]:
means = df.groupby("age_first_code_cut")["converted_comp"].mean()
s = df.groupby("age_first_code_cut")["converted_comp"].std()
n = df.groupby("age_first_code_cut")["converted_comp"].count()

numerator = means.loc["child"] - means.loc["adult"]
denominator = np.sqrt(s.loc["child"]**2/n.loc["child"] + s.loc["adult"]**2/n.loc["adult"])
t_stat = numerator / denominator

print(t_stat)

1.8699313316221844


**Step 3:** Calculating the p-value

Degrees of freedom: maximum number of logically independent values in the data sample

In [17]:
degrees_of_freedom = n.loc["child"] + n.loc["adult"] - 2 # -2 because we have the means of both groups as logically independent values

from scipy.stats import t
p_value = 1 - t.cdf(t_stat, df=degrees_of_freedom)
print(p_value)

0.030811302165157595


**Step 4:** Comparing the p-value against the significance level

In [ ]:
alpha = 0.05
if p_value <= alpha:
    print("Null hypothesis rejected")
else:
    print("Failed to reject the null hypothesis")

# Paired t-tests

In [1]:
# Calculate the differences from 2012 to 2016
sample_dem_data['diff'] = sample_dem_data['dem_percent_12'] - sample_dem_data['dem_percent_16']

# Find the mean of the diff column
xbar_diff = sample_dem_data['diff'].mean()

# Find the standard deviation of the diff column
s_diff = sample_dem_data['diff'].std()

# Plot a histogram of diff with 20 bins
sample_dem_data['diff'].hist(bins=20)
plt.show()

NameError: name 'sample_dem_data' is not defined

In [2]:
# Conduct a t-test on diff
test_results = pingouin.ttest(x=sample_dem_data['diff'], 
                              y=0, 
                              alternative="two-sided")

# Conduct a paired t-test on dem_percent_12 and dem_percent_16
paired_test_results = pingouin.ttest(x=sample_dem_data['dem_percent_12'], 
                                     y=sample_dem_data['dem_percent_16'],
                                     paired=True,
                                     alternative="two-sided")
                              
# Print the paired test results
print(paired_test_results)

NameError: name 'pingouin' is not defined

# ANOVA tests

In [ ]:
# Calculate the mean pack_price for each shipment_mode
xbar_pack_by_mode = late_shipments.groupby("shipment_mode")['pack_price'].mean()

# Calculate the standard deviation of the pack_price for each shipment_mode
s_pack_by_mode = late_shipments.groupby("shipment_mode")['pack_price'].std()

# Boxplot of shipment_mode vs. pack_price
sns.boxplot(x="pack_price", y="shipment_mode", data=late_shipments)
plt.show()

# Run an ANOVA for pack_price across shipment_mode
anova_results = pingouin.anova(data=late_shipments,
                               dv="pack_price",
                               between="shipment_mode")

# Print anova_results
print(anova_results)

In [ ]:
# Perform a pairwise t-test on pack price, grouped by shipment mode
pairwise_results = pingouin.pairwise_tests(data=late_shipments, 
                                           dv="pack_price",
                                           between="shipment_mode",
                                           padjust="none")

# Print pairwise_results
print(pairwise_results)

In [ ]:
# Modify the pairwise t-tests to use Bonferroni p-value adjustment
pairwise_results = pingouin.pairwise_tests(data=late_shipments, 
                                           dv="pack_price",
                                           between="shipment_mode",
                                           padjust="bonf")

# Print pairwise_results
print(pairwise_results)